In [1]:
!unzip "/content/drive/My Drive/driving_dataset.zip"

Streaming output truncated to the last 5000 lines.
  inflating: driving_dataset/550.jpg  
  inflating: driving_dataset/5500.jpg  
  inflating: driving_dataset/5501.jpg  
  inflating: driving_dataset/5502.jpg  
  inflating: driving_dataset/5503.jpg  
  inflating: driving_dataset/5504.jpg  
  inflating: driving_dataset/5505.jpg  
  inflating: driving_dataset/5506.jpg  
  inflating: driving_dataset/5507.jpg  
  inflating: driving_dataset/5508.jpg  
  inflating: driving_dataset/5509.jpg  
  inflating: driving_dataset/551.jpg  
  inflating: driving_dataset/5510.jpg  
  inflating: driving_dataset/5511.jpg  
  inflating: driving_dataset/5512.jpg  
  inflating: driving_dataset/5513.jpg  
  inflating: driving_dataset/5514.jpg  
  inflating: driving_dataset/5515.jpg  
  inflating: driving_dataset/5516.jpg  
  inflating: driving_dataset/5517.jpg  
  inflating: driving_dataset/5518.jpg  
  inflating: driving_dataset/5519.jpg  
  inflating: driving_dataset/552.jpg  
  inflating: driving_dataset/552

In [2]:
import pandas
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
import keras

data = pandas.read_csv('/content/drive/My Drive/data.txt',header = None,sep=' ')
data = data.values

In [3]:
x = data[:,0]
y = data[:,-1]

print(x[0])

0.jpg


In [4]:
from keras.preprocessing import image
def Load_Batch(batch_size, i, x, y):
  xd = []
  yd = []
  pointer = 0
  if(i == (len(x)//batch_size)-1):
    batch_size = len(x)%batch_size
  while pointer<batch_size:
    img = image.load_img('/content/driving_dataset/'+x[(pointer + batch_size*i)%l],color_mode='rgb',target_size=(160,320) )
    xd.append(image.img_to_array(img)/255.0)
    yd.append(y[(pointer + batch_size*i)%l]*np.pi/180.0)
    pointer = pointer + 1
  return np.array(xd), np.array(yd)


In [5]:
print(y[20000])

1.01


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print("Shape of X_train",X_train.shape)
print("Shape of X_test", X_test.shape)
print("Shape of y_train",y_train.shape)
print("Shape of y_test", y_test.shape)

Shape of X_train (36324,)
Shape of X_test (9082,)
Shape of y_train (36324,)
Shape of y_test (9082,)


In [7]:
from keras.layers import Conv2D,Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, MaxPool2D , Cropping2D
from keras.optimizers import Adam
from keras.models import Sequential
from keras import regularizers
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Lambda


In [8]:
model = Sequential()

model.add(Cropping2D(cropping=((80,0), (0, 0)), input_shape=(160,320,3)))
model.add(BatchNormalization())
model.add(Lambda(lambda x: tf.image.resize(x, (66,200)))) 

model.add(Conv2D(24,(5,5),activation='relu',kernel_regularizer=regularizers.l2(0.001),strides = (2,2)))
model.add(BatchNormalization())

model.add(Conv2D(36,(5,5),activation='relu',kernel_regularizer=regularizers.l2(0.001),strides = (2,2)))
model.add(BatchNormalization())

model.add(Conv2D(48,(5,5),activation='relu',kernel_regularizer=regularizers.l2(0.001),strides = (2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64,(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())

model.add(Conv2D(128,(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())


model.add(Flatten())

model.add(Dense(1164,activation='relu', kernel_regularizer=regularizers.l2(0.001)))

model.add(Dense(200,activation='relu', kernel_regularizer=regularizers.l2(0.001)))

model.add(Dense(50,activation='relu', kernel_regularizer=regularizers.l2(0.001)))

model.add(Dense(10,activation='relu', kernel_regularizer=regularizers.l2(0.001)))

model.add(Dense(1,activation='tanh', kernel_regularizer=regularizers.l2(0.001)))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d (Cropping2D)      (None, 80, 320, 3)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 80, 320, 3)        12        
_________________________________________________________________
lambda (Lambda)              (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 31, 98, 24)        1824      
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 98, 24)        96        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 47, 36)        1

In [9]:
model.compile(optimizer=Adam(0.001), loss = 'mse', metrics = ['mae'])

In [10]:
epochs = 5
batch_size = 128
num = len(X_train)
c=0
l = len(X_test)
for epoch in range(epochs):
    for i in range(int(num//batch_size)):
        xs, ys = Load_Batch(batch_size,i,X_train,y_train)
        ys.reshape(-1,1)

        model.fit(xs,ys,verbose=0)

        if i%10 == 0:
            score_train = model.evaluate(xs,ys,verbose=0)
            xs1, ys1 = Load_Batch(batch_size,c,X_test,y_test)
            c+=1
            ys1.reshape(-1,1)
            score_test = model.evaluate(xs1,ys1,verbose=0)
            print("Epoch:%d Step:%d Train loss:%f Train mae:%f Test loss:%f Test mae:%f\n"%(epoch+1,i,score_train[0],score_train[1],score_test[0],score_test[1]))

        if i%100 == 0:
            model.save_weights("/content/drive/My Drive/final_weights.h5")



Epoch:1 Step:0 Train loss:2.405058 Train mae:0.250804 Test loss:2.428976 Test mae:0.233027

Epoch:1 Step:10 Train loss:2.288906 Train mae:0.319854 Test loss:2.198813 Test mae:0.254552

Epoch:1 Step:20 Train loss:2.128764 Train mae:0.356052 Test loss:2.043459 Test mae:0.306140

Epoch:1 Step:30 Train loss:1.477479 Train mae:0.263183 Test loss:1.537373 Test mae:0.288553

Epoch:1 Step:40 Train loss:1.234055 Train mae:0.265191 Test loss:1.228333 Test mae:0.275946

Epoch:1 Step:50 Train loss:1.162330 Train mae:0.307706 Test loss:1.008902 Test mae:0.282263

Epoch:1 Step:60 Train loss:1.296011 Train mae:0.347575 Test loss:0.847166 Test mae:0.269081

Epoch:1 Step:70 Train loss:0.733075 Train mae:0.249746 Test loss:0.979364 Test mae:0.318621

Epoch:1 Step:80 Train loss:0.635205 Train mae:0.232424 Test loss:0.669230 Test mae:0.227884

Epoch:1 Step:90 Train loss:0.568775 Train mae:0.211672 Test loss:0.576039 Test mae:0.244286

Epoch:1 Step:100 Train loss:0.429260 Train mae:0.179683 Test loss:0.516